<a href="https://colab.research.google.com/github/Almasmm/Automated-Schedule-Generation/blob/main/Genetic_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import random

# Load the dataset
file_path = "/content/Extracted data from Actual Schedule for Compter science last year groups.xlsx"
df_allocated_times = pd.read_excel(file_path, sheet_name="Actual_Schedule_Allocated_Time")

def initialize_population(size, courses, time_slots):
    """Initialize population with random schedules adhering to allocated time slots."""
    population = []
    for _ in range(size):
        individual = {course: random.choice(time_slots) for course in courses}
        population.append(individual)
    return population

def fitness(schedule):
    """Evaluate schedule fitness based on constraints (minimizing idle time)."""
    idle_time = 0
    days = {"Monday": [], "Tuesday": [], "Wednesday": [], "Friday": [], "Saturday": []}

    for course, time in schedule.items():
        day, slot = time.split()
        if day in days:
            days[day].append(int(slot.split(':')[0]))

    for day, slots in days.items():
        if slots:
            slots.sort()
            idle_time += sum(slots[i+1] - slots[i] - 1 for i in range(len(slots)-1))

    return 1 / (1 + idle_time)  # Higher fitness for lower idle time

def selection(population):
    """Select top individuals using tournament selection."""
    selected = random.sample(population, k=5)
    selected.sort(key=lambda x: fitness(x), reverse=True)
    return selected[:2]

def crossover(parent1, parent2):
    """Perform crossover to create new offspring."""
    point = len(parent1) // 2
    child1 = {**dict(list(parent1.items())[:point]), **dict(list(parent2.items())[point:])}
    child2 = {**dict(list(parent2.items())[:point]), **dict(list(parent1.items())[point:])}
    return child1, child2

def mutate(individual, time_slots):
    """Randomly mutate a gene (change a course's time slot)."""
    course = random.choice(list(individual.keys()))
    individual[course] = random.choice(time_slots)
    return individual

def genetic_algorithm(iterations=100, pop_size=50):
    """Main genetic algorithm loop."""
    courses = list(df_allocated_times['Course'])
    time_slots = list(df_allocated_times['Allocated Time'])

    population = initialize_population(pop_size, courses, time_slots)

    for _ in range(iterations):
        new_population = []
        for _ in range(pop_size // 2):
            parent1, parent2 = selection(population)
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([mutate(child1, time_slots), mutate(child2, time_slots)])
        population = new_population

    best_schedule = max(population, key=fitness)
    return best_schedule

# Run algorithm
best_generated_schedule = genetic_algorithm()
print(best_generated_schedule)


ValueError: Worksheet named 'Actual_Schedule_Allocated_Time' not found